In [13]:
import socket
import sys
import platform
import os
from scapy.all import sniff, wrpcap
from config import constants

In [14]:
CATEGORIES = constants.CATEGORIES
print(CATEGORIES)

['cloud_service', 'e-mail', 'games', 'information', 'music', 'social_network', 'video_hosting']


In [15]:
DOMAINS = {
    'music': ["spotify.com", "open.spotify.com", "music.yandex.ru", "mc.yandex.ru", "music.apple.com", "zaycev.net"],
    'social_network': ["vk.com", "ok.ru", "web.telegram.org", "telegram.org", "t.me"],
    'video_hosting': ["twitch.tv", "youtube.com", "rutube.ru", "vkvideo.ru", "kinopoisk.ru", "hd.kinopoisk.ru", "ivi.ru", "premier.one"],
    'cloud_service': ["disk.yandex.ru", "cloud.mail.ru", "drive.google.com", "onedrive.live.com"],
    'e-mail': ["mail.google.com", "mail.yandex.ru", "mail.ru", "outlook.live.com", "gmail.com", "vk.mail.ru"],
    'information': ["wikipedia.org", "nstu.ru", "ngs.ru", "gismeteo.ru", "habr.com", "tass.ru", "hse.ru"]
}

In [16]:
CURRENT_CATEGORY = 'video_hosting'
DURATION = 600

In [17]:
print(f"Сбор трафика для {CURRENT_CATEGORY} с продолжительностью {DURATION}c.")

Сбор трафика для video_hosting с продолжительностью 600c.


In [18]:
system = platform.system().lower()

if not system:
    exit(1)

print(f"Текущая ОС {system}")

Текущая ОС windows


In [19]:
if system == "windows":
    INTERFACE = "Беспроводная сеть"
else:
    INTERFACE = "en0"

print(INTERFACE)

Беспроводная сеть


In [20]:
def resolve_domains(domains):
    """Разрешает домены в список всех IP-адресов (IPv4 и IPv6)."""
    ip_list = []
    for domain in domains:
        try:
            # Запрашиваем все адреса
            addr_info = socket.getaddrinfo(
                domain,
                None,
                family=0,
                type=socket.SOCK_STREAM,
                flags=socket.AI_ALL | socket.AI_V4MAPPED
            )
            # Извлекаем уникальные IP-адреса
            ip_set = set()
            for info in addr_info:
                ip = info[4][0]
                ip_set.add(ip)
            ip_list.extend(ip_set)

            print(f"Разрешен домен {domain} -> {list(ip_set)}")
        except socket.gaierror:
            print(f"Не удалось разрешить домен {domain}")
    return list(set(ip_list))  # Удаляем дубликаты

In [21]:
def create_bpf_filter():
    # Получение ip адресов по доменному имени
    ip_list = resolve_domains(DOMAINS.get(CURRENT_CATEGORY))
    if not ip_list:
        return ""

    ip_filter = " or ".join(f"host {ip}" for ip in ip_list)

    return f"{ip_filter}"

In [22]:
def start_capture(interface, output_file, duration=60):
    """Захват трафика и сохранение в PCAP-файл."""
    packets = []

    # Создание BPF-фильтра
    bpf_filter = create_bpf_filter()

    print(bpf_filter)

    if not bpf_filter:
        print("Не удалось создать фильтр для захвата трафика.")
        sys.exit(1)

    print(f"BPF-фильтр: {bpf_filter}")
    print(f"Начало захвата трафика на интерфейсе {interface}...")

    def packet_callback(packet):
        packets.append(packet)
        print(f"Захвачен пакет: {packet.summary()}")

    try:
        # Захват трафика
        sniff(iface=interface, prn=packet_callback, timeout=duration, filter=bpf_filter)

        # Сохранение пакетов в PCAP-файл
        print(f"Сохранение {len(packets)} пакетов в файл {output_file}...")
        wrpcap(output_file, packets)
        print(f"Захват завершен. Файл сохранен: {output_file}")

    except Exception as e:
        print(f"Ошибка при захвате трафика: {e}")
        sys.exit(1)

In [23]:
RAW_PATH = f'raw/{CURRENT_CATEGORY}'
pcap_files = [f for f in os.listdir(RAW_PATH) if f.endswith('.pcap')]
FILENAME = f"{len(pcap_files) + 1}_{system}_{DURATION}.pcap"
OUTPUT_FILE = f"{RAW_PATH}/{FILENAME}"

In [24]:
start_capture(INTERFACE, OUTPUT_FILE, DURATION)

Разрешен домен twitch.tv -> ['151.101.66.167', '151.101.130.167', '151.101.2.167', '151.101.194.167']
Разрешен домен youtube.com -> ['142.250.150.190', '142.250.150.136', '2a00:1450:4026:804::200e', '142.250.150.93', '142.250.150.91']
Разрешен домен rutube.ru -> ['178.248.233.148']
Разрешен домен vkvideo.ru -> ['93.186.225.194', '87.240.129.133', '87.240.132.78', '87.240.132.72', '87.240.137.164', '87.240.132.67']
Разрешен домен kinopoisk.ru -> ['213.180.199.9', '2a02:6b8::473']
Разрешен домен hd.kinopoisk.ru -> ['213.180.199.9', '2a02:6b8::473']
Разрешен домен ivi.ru -> ['80.67.43.33', '80.67.43.65', '80.67.43.1']
Разрешен домен premier.one -> ['194.190.77.89']
host 151.101.2.167 or host 151.101.194.167 or host 87.240.129.133 or host 142.250.150.91 or host 151.101.66.167 or host 93.186.225.194 or host 80.67.43.1 or host 2a02:6b8::473 or host 87.240.132.72 or host 142.250.150.93 or host 151.101.130.167 or host 178.248.233.148 or host 142.250.150.190 or host 213.180.199.9 or host 194.19